In [ ]:
%pip install -U langchain-community langchain_chroma pypdf gigachat -q

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chat_models.gigachat import GigaChat
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [ ]:
import os
giga_key = os.getenv("GIGACHAT_API_KEY")

In [ ]:
loader = PyPDFLoader('Machine_Learning_System_Design.pdf')
pages = loader.load_and_split()
pages[:1]

[Document(metadata={'source': 'Machine_Learning_System_Design.pdf', 'page': 1}, page_content='MEAP Edition\nManning Early Access Program\nMachine Learning System Design\nWith end-to-end examples\nVersion 12\nCopyright 2024 Manning Publications\nFor more information on this and other Manning titles go to manning.com.\n© Manning Publications Co. To comment go to liveBook\nLicensed to Valerii Babushkin <venheads@yandex.ru>')]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(pages)

In [ ]:
giga = GigaChat(credentials=giga_key,
                    model="GigaChat-Pro", timeout=30, verify_ssl_certs=False)

<ipython-input-10-5dc48a8bb93f>:2: LangChainDeprecationWarning: The class `GigaChat` was deprecated in LangChain 0.3.5 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-gigachat package and should be used instead. To use it run `pip install -U :class:`~langchain-gigachat` and import as `from :class:`~langchain_gigachat import GigaChat``.
  giga = GigaChat(credentials=giga_key,


In [ ]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
CHUNK_SIZE = 1000
CHUNK_OVERLAP_SCALE = 0.1

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " "], # separates either on words or paragraphs
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_SIZE*CHUNK_OVERLAP_SCALE,
)

In [ ]:
splits = splitter.split_documents(documents)

In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    encode_kwargs={"normalize_embeddings": True}
)

<ipython-input-14-6ef8cc4d2d0b>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | giga
    | StrOutputParser()
)

In [ ]:
print(rag_chain.invoke("Write design document for a sales forecasting ML system"))

A design document for a sales forecasting ML system should outline the steps involved in preparing the data for modeling, including data cleaning, feature engineering, and data normalization. Additionally, it should specify the types of models that will be used, such as baseline methods (moving average, exponential smoothing) and machine learning algorithms (decision trees, random forests, etc.). Finally, the document should address potential risks and considerations, such as the impact of different forecast accuracy levels and the need for scalability and robustness against unexpected events.


# Save vector store

In [4]:
%pip install -U langchain-community pypdf faiss-gpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.7 MB/s eta 0:00:00


In [5]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

class DocumentEmbedder:
    def __init__(self, file_path, chunk_size=1000, chunk_overlap=200, embedding_model_name="sentence-transformers/all-mpnet-base-v2"):
        self.file_path = file_path
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.embedding_model = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            multi_process=True,
            encode_kwargs={"normalize_embeddings": True}
        )

    def load_and_split(self):
        loader = PyPDFLoader(self.file_path)
        pages = loader.load_and_split()

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )
        return text_splitter.split_documents(pages)

    def create_embeddings(self):
        documents = self.load_and_split()
        return FAISS.from_documents(documents, self.embedding_model)

    def save_embeddings(self, vectorstore, path):
        vectorstore.save_local(path)

def main():
    pdf_path = "/content/Machine_Learning_System_Design.pdf"
    vectorstore_path = "vectorstore"

    embedder = DocumentEmbedder(pdf_path)

    print("Creating embeddings...")
    vectorstore = embedder.create_embeddings()

    print("Saving embeddings...")
    embedder.save_embeddings(vectorstore, vectorstore_path)

    print(f"Embeddings saved to {vectorstore_path}")


main()

Creating embeddings...
Saving embeddings...
Embeddings saved to vectorstore
